In [10]:
from octopus import OctopusClient

import networkx as nx

In [11]:
def build_nx_graph():
    """
    Builds and returns networkx graph for Movies.

    Parameters : 
    None

    Returns : 
    networkx graph object
        A networkx graph object of Movies
    """
    raise NotImplementedError

In [12]:
def task_1_nx(nx_graph):
    """
    Using networkx library, find the actor named "Tom Hanks".

    Parameters:
    nx_graph : networkx object
       Movies graph
    
    Returns :
    None
    """
    raise NotImplementedError

In [13]:
def task_1_octopus(oc):
    """
    Using Octopus, find the actor named "Tom Hanks".

    Parameters:
    oc : Octopus Client
        Octopus Client
    
    Returns :
    None
    """
    query = None
    execution = oc.execute(query, 1)
    execution.poll()
    print(execution.output.fetch_all())

In [14]:
def task_2_nx(nx_graph):
    """
    Using networkx library, find all movies released in the 90s and print their titles.

    Parameters:
    nx_graph : networkx object
       Movies graph
    
    Returns :
    None
    """
    raise NotImplementedError
    

In [15]:
def task_2_octopus(oc):
    """
    Using Octopus, find all movies released in the 90s and print their titles.

    Parameters:
    oc : Octopus Client
        Octopus Client
        
    Returns :
    None
    """
    query = None
    execution = oc.execute(query, 1)
    execution.poll()
    print(execution.output.fetch_all())  

In [16]:
def task_3_nx(nx_graph):
    """
    Using networkx library, find how people are related to the movie "Cloud Atlas".

    Parameters:
    nx_graph : networkx object
       Movies graph
    
    Returns :
    None
    """
    raise NotImplementedError


In [17]:
def task_3_octopus(nx_graph):
    """
    Using Octopus, find how people are related to the movie "Cloud Atlas".

    Parameters:
     oc : Octopus Client
        Octopus Client
    
    Returns :
    None
    """
    query = None
    execution = oc.execute(query, 1)
    execution.poll()
    print(execution.output.fetch_all())  

In [18]:
if __name__ == "__main__":
    # Build networkx graph.
    # nx_graph = build_nx_graph()

    # Instantiate Octopus Client
    oc = OctopusClient("localhost", "8000", "neo4j", "password")

    # Task-1
    # print("Task-1: Find the actor named \"Tom Hanks\"")
    # print("networkx:")
    # task_1_nx(nx_graph)
    # print("Octopus:")
    # task_1_octopus(oc)

    # Task-2
    # print("Task-2: Find movies released in the 90s and return their titles")
    # print("networkx:")
    # task_2_nx(nx_graph)
    # print("Octopus:")
    # task_2_octopus(oc)

    # Task-3
    # print("Task-3: How people are related to \"Cloud Atlas\"?")
    # print("networkx:")
    # task_3_nx(nx_graph)
    # print("Octopus:")
    # task_3_octopus(oc)    
    